In [1]:
# to import modules locally without having installed the entire package
# http://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

In [2]:
import subprocess
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook')

/Users/nicolasdrizard/anaconda/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
# database parameters
ts_length = 100
data_dir = '../db_files'
db_name = 'default'
dir_path = data_dir + '/' + db_name + '/'

In [20]:
# clear file system for testing
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
filelist = [dir_path + f for f in os.listdir(dir_path)]
for f in filelist:
    os.remove(f)

In [21]:
from timeseries import *

def tsmaker(m, s, j):
    '''
    Helper function: randomly generates a time series for testing.

    Parameters
    ----------
    m : float
        Mean value for generating time series data
    s : float
        Standard deviation value for generating time series data
    j : float
        Quantifies the "jitter" to add to the time series data

    Returns
    -------
    A time series and associated meta data.
    '''

    # generate metadata
    meta = {}
    meta['order'] = int(np.random.choice(
        [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]))
    meta['blarg'] = int(np.random.choice([1, 2]))

    # generate time series data
    t = np.arange(0.0, 1.0, 0.01)
    v = norm.pdf(t, m, s) + j * np.random.randn(ts_length)

    # return time series and metadata
    return meta, TimeSeries(t, v)

In [22]:
# generate sample time series
num_ts = 50
mus = np.random.uniform(low=0.0, high=1.0, size=num_ts)
sigs = np.random.uniform(low=0.05, high=0.4, size=num_ts)
jits = np.random.uniform(low=0.05, high=0.2, size=num_ts)

# initialize dictionaries for time series and their metadata
primary_keys = []
tsdict = {}
metadict = {}

# fill dictionaries with randomly generated entries for database
for i, m, s, j in zip(range(num_ts), mus, sigs, jits):
    meta, tsrs = tsmaker(m, s, j)  # generate data
    pk = "ts-{}".format(i)  # generate primary key
    primary_keys.append(pk) # keep track of all primary keys
    tsdict[pk] = tsrs  # store time series data
    metadict[pk] = meta  # store metadata
    
# to assist with later testing
ts_keys = sorted(tsdict.keys())
    
# randomly choose time series as vantage points
num_vps = 5
random_vps = np.random.choice(range(num_ts), size=num_vps, replace=False)
vpkeys = sorted(['ts-{}'.format(i) for i in random_vps])

In [23]:
from tsdb import *

In [24]:
identity = lambda x: x

# index: 1 is binary tree index, 2 is bitmap index
schema = {
  'pk': {'type': 'str', 'convert': identity, 'index': None, 'values': None},
  'ts': {'type': 'int', 'convert': identity, 'index': None, 'values': None},
  'order': {'type': 'int', 'convert': int, 'index': 1, 'values': None},
  'blarg': {'type': 'int', 'convert': int, 'index': 1, 'values': None},
  'useless': {'type': 'int', 'convert': identity, 'index': 1, 'values': None},
  'mean': {'type': 'float', 'convert': float, 'index': 1, 'values': None},
  'std': {'type': 'float', 'convert': float, 'index': 1, 'values': None},
  'vp': {'type': 'bool', 'convert': bool, 'index': 2, 'values': [True, False]},
  'deleted': {'type': 'bool', 'convert': bool, 'index': 2, 'values': [True, False]}
}

In [25]:
# Creating db
db = PersistentDB(schema, 'pk', 100, db_name=db_name, data_dir=data_dir, commit_step=15)

In [26]:
# Inserting ts
for pk, t in tsdict.items():
    db.insert_ts(pk, t)

COMMITTING TO DISK
COMMITTING TO DISK
COMMITTING TO DISK


In [27]:
# Inserting meta
for pk, m in metadict.items():
    db.upsert_meta(pk, m)

In [11]:
print(len(db.pks.keys()))
print(db.pks.keys())
print(db.pks.values())

50
dict_keys(['ts-7', 'ts-21', 'ts-0', 'ts-1', 'ts-13', 'ts-22', 'ts-34', 'ts-48', 'ts-9', 'ts-38', 'ts-10', 'ts-28', 'ts-11', 'ts-8', 'ts-30', 'ts-3', 'ts-25', 'ts-35', 'ts-14', 'ts-46', 'ts-5', 'ts-18', 'ts-43', 'ts-41', 'ts-47', 'ts-27', 'ts-2', 'ts-20', 'ts-26', 'ts-39', 'ts-42', 'ts-15', 'ts-12', 'ts-45', 'ts-44', 'ts-23', 'ts-49', 'ts-31', 'ts-32', 'ts-17', 'ts-19', 'ts-6', 'ts-24', 'ts-29', 'ts-4', 'ts-37', 'ts-16', 'ts-33', 'ts-36', 'ts-40'])
dict_values([(4, 0), (1604, 25), (3204, 50), (4804, 75), (6404, 100), (8004, 125), (9604, 150), (11204, 175), (12804, 200), (33604, 525), (54404, 850), (56004, 875), (17604, 275), (19204, 300), (20804, 325), (22404, 350), (24004, 375), (25604, 400), (27204, 425), (28804, 450), (30404, 475), (32004, 500), (49604, 775), (35204, 550), (36804, 575), (38404, 600), (40004, 625), (41604, 650), (43204, 675), (44804, 700), (46404, 725), (48004, 750), (16004, 250), (62404, 975), (51204, 800), (52804, 825), (14404, 225), (57604, 900), (59204, 925), (

In [12]:
# Inspecting log
print('LOG')
print(len(db.pks.log))
print(db.pks.log)
print('INDEX')
print(len(db.pks.index))
print(db.pks.index)

LOG
51
{'ts-7': (4, 0), 'ts-21': (1604, 25), 'ts-0': (3204, 50), 'ts-1': (4804, 75), 'ts-13': (6404, 100), 'ts-22': (8004, 125), 'ts-34': (9604, 150), 'ts-2': (40004, 625), 'ts-48': (11204, 175), 'ts-9': (12804, 200), 'ts-38': (33604, 525), 'ts-10': (54404, 850), 'ts-28': (56004, 875), 'ts-11': (17604, 275), 'ts-8': (19204, 300), 'ts-30': (20804, 325), 'ts-3': (22404, 350), 'ts-25': (24004, 375), 'ts-35': (25604, 400), 'ts-14': (27204, 425), 'ts-46': (28804, 450), 'ts-5': (30404, 475), 'ts-18': (32004, 500), 'ts-43': (49604, 775), 'ts-41': (35204, 550), 'ts-47': (36804, 575), 'ts-27': (38404, 600), '$COMMITED$': False, 'ts-20': (41604, 650), 'ts-26': (43204, 675), 'ts-39': (44804, 700), 'ts-42': (46404, 725), 'ts-15': (48004, 750), 'ts-12': (16004, 250), 'ts-45': (62404, 975), 'ts-44': (51204, 800), 'ts-23': (52804, 825), 'ts-49': (14404, 225), 'ts-31': (57604, 900), 'ts-32': (59204, 925), 'ts-17': (60804, 950), 'ts-19': (64004, 1000), 'ts-6': (65604, 1025), 'ts-24': (67204, 1050), 'ts

In [13]:
# Loading index from file
file = db.pks.file
with open(file, "rb", buffering=0) as fd:
    index_load = pickle.load(fd)

In [14]:
len(index_load)

45

In [15]:
# Closing db
db.close()

In [16]:
# Checking error checked
db._assert_not_closed()

ValueError: Database closed.

In [17]:
db = PersistentDB(schema, 'pk', 100, db_name=db_name, data_dir=data_dir, commit_step=15)

In [18]:
print('LOG')
print(len(db.pks.log))
print(db.pks.log)
print('INDEX')
print(len(db.pks.index))
print(db.pks.index)

LOG
51
{'ts-7': (4, 0), 'ts-21': (1604, 25), 'ts-0': (3204, 50), 'ts-1': (4804, 75), 'ts-13': (6404, 100), 'ts-22': (8004, 125), 'ts-34': (9604, 150), '$COMMITED$': True, 'ts-48': (11204, 175), 'ts-9': (12804, 200), 'ts-38': (33604, 525), 'ts-49': (14404, 225), 'ts-28': (56004, 875), 'ts-11': (17604, 275), 'ts-8': (19204, 300), 'ts-30': (20804, 325), 'ts-3': (22404, 350), 'ts-25': (24004, 375), 'ts-35': (25604, 400), 'ts-14': (27204, 425), 'ts-46': (28804, 450), 'ts-5': (30404, 475), 'ts-18': (32004, 500), 'ts-43': (49604, 775), 'ts-41': (35204, 550), 'ts-47': (36804, 575), 'ts-27': (38404, 600), 'ts-2': (40004, 625), 'ts-20': (41604, 650), 'ts-26': (43204, 675), 'ts-39': (44804, 700), 'ts-42': (46404, 725), 'ts-15': (48004, 750), 'ts-12': (16004, 250), 'ts-45': (62404, 975), 'ts-44': (51204, 800), 'ts-23': (52804, 825), 'ts-10': (54404, 850), 'ts-31': (57604, 900), 'ts-32': (59204, 925), 'ts-17': (60804, 950), 'ts-19': (64004, 1000), 'ts-6': (65604, 1025), 'ts-24': (67204, 1050), 'ts-

In [19]:
print(len(db.pks.keys()))
print(db.pks.keys())
print(db.pks.values())

50
dict_keys(['ts-7', 'ts-21', 'ts-0', 'ts-1', 'ts-13', 'ts-22', 'ts-34', 'ts-48', 'ts-9', 'ts-10', 'ts-28', 'ts-11', 'ts-8', 'ts-30', 'ts-3', 'ts-25', 'ts-35', 'ts-14', 'ts-46', 'ts-5', 'ts-18', 'ts-38', 'ts-41', 'ts-47', 'ts-27', 'ts-2', 'ts-20', 'ts-26', 'ts-39', 'ts-42', 'ts-15', 'ts-12', 'ts-43', 'ts-44', 'ts-23', 'ts-49', 'ts-31', 'ts-32', 'ts-17', 'ts-45', 'ts-19', 'ts-6', 'ts-24', 'ts-29', 'ts-4', 'ts-37', 'ts-16', 'ts-33', 'ts-36', 'ts-40'])
dict_values([(4, 0), (1604, 25), (3204, 50), (4804, 75), (6404, 100), (8004, 125), (9604, 150), (11204, 175), (12804, 200), (54404, 850), (56004, 875), (17604, 275), (19204, 300), (20804, 325), (22404, 350), (24004, 375), (25604, 400), (27204, 425), (28804, 450), (30404, 475), (32004, 500), (33604, 525), (35204, 550), (36804, 575), (38404, 600), (40004, 625), (41604, 650), (43204, 675), (44804, 700), (46404, 725), (48004, 750), (16004, 250), (49604, 775), (51204, 800), (52804, 825), (14404, 225), (57604, 900), (59204, 925), (60804, 950), (